## Create bulls/bears datasets

In [50]:
import json
import datetime
import pandas as pd
import numpy as np
import gc
import glob
import json
import os
import preprocessor as p
import mysql.connector as sql
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
import traceback
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [17]:
pd.options.display.float_format = '{:.6f}'.format
pd.options.display.max_colwidth = 400
p.set_options(p.OPT.URL, p.OPT.MENTION)
analyzer = SentimentIntensityAnalyzer()

In [55]:
ssh_username = os.getenv('AUTOMLPREDICTOR_DB_SSH_USER')
ssh_password = os.getenv('AUTOMLPREDICTOR_DB_SSH_PASSWORD')

db_host = os.getenv('AUTOMLPREDICTOR_DB_SERVER_IP', '127.0.0.1')
db_user = 'root'
db_password = os.getenv('AUTOMLPREDICTOR_DB_SQL_PASSWORD')
db_port = 3306
db = 'automlpredictor_db_dashboard'
use_ssh = True

In [10]:
data_path='./data/twitter'

In [40]:
def get_friends_followers(leaders_path, friends_path, followers_path):
    df_leaders = pd.read_json(leaders_path, lines=True, orient=str)
    df_leaders['id'] = df_leaders[['id']].astype(np.int64)
    
    df_friends = pd.read_json(friends_path, lines=True, orient=str) # .reset_index(drop=True).set_index(['id'])
    df_followers = pd.read_json(followers_path, lines=True, orient=str).reset_index(drop=True).set_index(['id'])
    
    
    df_intersection = pd.merge(
        df_friends[['id']], df_followers, left_on=['id'], 
        right_index=True, how='inner', sort=False, suffixes=['', '_']
    )
    
    df_union = pd.concat([df_intersection, df_leaders]).drop_duplicates().reset_index(drop=True).set_index(['id'])
    return df_intersection

In [41]:
df_bears = get_friends_followers(f'{data_path}/bears_20180912.json',
                                 f'{data_path}/bears_friends_20180916.json', 
                                 f'{data_path}/bears_followers_20180914.json')    

In [43]:
df_bulls = get_friends_followers(f'{data_path}/bulls_20180912.json',
                                 f'{data_path}/bulls_friends_20180915.json', 
                                 f'{data_path}/bulls_followers_20180916.json')    

In [45]:
df_bulls.to_json(
    f'{data_path}/extended_bulls_20180912.json', orient='records', lines=True
)

In [46]:
df_bears.to_json(
    f'{data_path}/extended_bears_20180912.json', orient='records', lines=True
)

In [64]:
df_bulls_bears = pd.concat(
    [df_bulls, df_bears], keys=['Bullish', 'Bearish',]
).reset_index().drop(['level_1'], axis=1).rename(
    {'level_0': 'sentiment'}, axis=1
).set_index(['screen_name'])

In [72]:
# df_bulls_bears

## Retrieve the posts to enrich

In [52]:
def query(use_ssh, q, db_host, db_user, db_password, db_port, db, ssh_username, ssh_password, charset='utf8mb4'):

    if use_ssh:
        with SSHTunnelForwarder(
                ssh_address_or_host=(db_host, 22),
                ssh_password=ssh_password,
                ssh_username=ssh_username,
                remote_bind_address=('127.0.0.1', db_port)
        ) as server:
            conn = sql.connect(host='127.0.0.1',
                               port=server.local_bind_port,
                               user=db_user,
                               passwd=db_password,
                               db=db,
                               charset=charset)
            response = pd.read_sql_query(q, conn)
            conn.close()
            return response
    else:
        conn = sql.connect(host=db_host,
                           port=db_port,
                           user=db_user,
                           passwd=db_password,
                           db=db,
                           charset=charset)
        response = pd.read_sql_query(q, conn)
        conn.close()
        return response

In [59]:
sql_posts_to_enrich = """
SELECT post_id, post_type, body, impact, link, user_name, 
created_at_epoch_ms, client_received_epoch_ms, 
sentiment_ml_model as original_ml_sentiment, 
sentiment_vader_normalized, 
sentiment_mixed as original_sentiment_mixed
FROM automlpredictor_db_dashboard.analysis_posts_sentiment
 WHERE created_at_epoch_ms >=(SELECT UNIX_TIMESTAMP(NOW())*1000-(14*24*3600000)) 
 AND post_type in ('twitter-topic', 'twitter-user');
"""

df_posts_to_enrich = query(
    use_ssh, sql_posts_to_enrich, db_host, db_user, db_password, db_port, db, 
    ssh_username, ssh_password
)


2018-09-18 11:37:51,092 - paramiko.transport - INFO - Connected (version 2.0, client OpenSSH_7.2p2)
2018-09-18 11:37:51,269 - paramiko.transport - INFO - Authentication (publickey) failed.
2018-09-18 11:37:51,307 - paramiko.transport - INFO - Connected (version 2.0, client OpenSSH_7.2p2)
2018-09-18 11:37:51,479 - paramiko.transport - INFO - Authentication (publickey) successful!


In [61]:
df_posts_to_enrich.to_json(
    f'{data_path}/twitter_posts_pre_enrichment_20180918.json.gz', 
    orient='records', lines=True, compression='gzip'
)

In [84]:
df_posts_to_enrich = pd.read_json(
    f'{data_path}/twitter_posts_pre_enrichment_20180918.json.gz', 
    lines=True, orient=str, compression='gzip'
)

In [77]:
df_enriched = pd.merge(
    df_posts_to_enrich, df_bulls_bears, left_on=['user_name'], 
    right_index=True, how='inner', sort=False
)# .rename(
#     {
#         'text': 'body'
#     }
# )
#         'favorite_count': 'likes.total',
#         'retweet_count': 'retweet.total',
#     }, axis=1
# )

In [81]:
# df_enriched

In [80]:
df_enriched.to_json(
    f'{data_path}/twitter_posts_post_enrichment_20180901_20180918.json.gz', 
    orient='records', lines=True, compression='gzip'
)

In [68]:
df_enriched_unknown_user = pd.merge(
    df_posts_to_enrich, df_bulls_bears, left_on=['user_name'], 
    right_index=True, how='left', sort=False
)

In [82]:
# df_enriched_unknown_user